# Simple Linear Model
By Gaetano Bonofiglio, Veronica Iovinella

## Introduction
We'll start by developing a simple linear model for classification of handwritten digits using MNIST data-set and then plot the results. This will later be compared with a Convolutional Neural Network for the same task.

### Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

tf.__version__

'0.9.0'

### Data Load
The MNIST data-set is about 12 MB and will be downloaded automatically if it is not located in the given path.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


The MNIST data-set has now been loaded and consists of 70.000 images and associated labels. The data-set is split into 3 mutually exclusive sub-sets.

In [3]:
print("Size of")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Size of
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000
